In [1]:
%run init.ipynb

ModuleNotFoundError: No module named 'matchzoo'

In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left=10,
    truncated_length_right=100,
    remove_stop_words=False
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11048.51it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5816.25it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 3465840.86it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11188.53it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5967.63it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 973584.52it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8884.23it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x10c5040b8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x135274dd8>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675}

In [5]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
histgram_callback = mz.dataloader.callbacks.Histogram(
    embedding_matrix, bin_size=30, hist_mode='LCH'
)
padding_callback = mz.models.DRMM.get_default_padding_callback()

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10,
    callbacks=[histgram_callback]
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    callback=padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    callbacks=[histgram_callback]
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [9]:
model = mz.models.DRMM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding'] = embedding_matrix
model.params['hist_bin_size'] = 30
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

DRMM(
  (embedding): Embedding(16675, 300)
  (attention): Attention(
    (linear): Linear(in_features=300, out_features=1, bias=False)
  )
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=30, out_features=10, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=1, bias=True)
      (1): Tanh()
    )
  )
  (out): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
  )
) 5003123


In [10]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [11]:
trainer.run()

[Iter-255 Loss-2.152]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5871 - normalized_discounted_cumulative_gain@5(0.0): 0.6444 - mean_average_precision(0.0): 0.6012



[Iter-510 Loss-1.789]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6062 - normalized_discounted_cumulative_gain@5(0.0): 0.6607 - mean_average_precision(0.0): 0.6218



[Iter-765 Loss-1.548]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6055 - normalized_discounted_cumulative_gain@5(0.0): 0.6598 - mean_average_precision(0.0): 0.6127



[Iter-1020 Loss-1.374]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6112 - normalized_discounted_cumulative_gain@5(0.0): 0.6639 - mean_average_precision(0.0): 0.6212



[Iter-1275 Loss-1.275]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6101 - normalized_discounted_cumulative_gain@5(0.0): 0.668 - mean_average_precision(0.0): 0.6232



[Iter-1530 Loss-1.233]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6188 - normalized_discounted_cumulative_gain@5(0.0): 0.67 - mean_average_precision(0.0): 0.6261



[Iter-1785 Loss-1.194]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6123 - normalized_discounted_cumulative_gain@5(0.0): 0.6601 - mean_average_precision(0.0): 0.6149



[Iter-2040 Loss-1.175]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6177 - normalized_discounted_cumulative_gain@5(0.0): 0.6679 - mean_average_precision(0.0): 0.6252



[Iter-2295 Loss-1.163]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6123 - normalized_discounted_cumulative_gain@5(0.0): 0.6623 - mean_average_precision(0.0): 0.6223



[Iter-2550 Loss-1.153]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6147 - normalized_discounted_cumulative_gain@5(0.0): 0.6622 - mean_average_precision(0.0): 0.6219

Cost time: 1394.7240190505981s
